In [1]:
import sqlalchemy
import numpy as np
import pandas as pd
import geopandas as gpd
from geoalchemy2 import Geometry
from geopandas_postgis import PostGIS
import pickle

In [2]:
sqlalchemy.__version__

'1.3.19'

In [3]:
from sqlalchemy import create_engine
engine = create_engine('')

In [79]:
df = gpd.read_file('sample_data.geojson')

In [3]:
# Opening pickle file
with open( "lucky_parking.p", "rb" ) as f:
  df = pickle.load(f)

In [10]:
df.postgis.to_postgis(con=engine, table_name='test', geometry='Point')

In [36]:
insp = sqlalchemy.inspect(engine)

In [37]:
db_list = insp.get_schema_names()

In [38]:
db_list

['information_schema', 'public', 'topology']

In [ ]:
for schema in db_list:
    print("schema: %s" % schema)
    for table_name in insp.get_table_names(schema=schema):
        for column in insp.get_columns(table_name, schema=schema):
            print("Column: %s" % column)

In [42]:
insp.get_table_names('public')

['spatial_ref_sys', 'test']

In [4]:
df.head()

,Ticket number,Meter Id,Marked Time,RP State Plate,Plate Expiry Date,VIN,Make,Body Style,Color,Location,...,Violation Description,Fine amount,Latitude,Longitude,Agency Description,Color Description,Body Style Description,Datetime,Plate Expiry Date Parsed,days_expired
2,1104803000,NaN,NaN,CA,201503.0,NaN,NISS,PA,BK,200 WORLD WAY,...,WHITE CURB,58.0,33.945212,-118.401151,NaN,NaN,NaN,2015-12-21 20:55:00,2015-03-01,295.0
3,1104820732,NaN,NaN,CA,NaN,NaN,ACUR,PA,WH,100 WORLD WAY,...,17104h,NaN,33.945212,-118.401009,NaN,NaN,NaN,2015-12-26 15:15:00,NaT,NaN
33,1108321723,NaN,NaN,CA,201610.0,NaN,HOND,PA,BK,10635 FRIAR ST,...,RED ZONE,93.0,34.185788,-118.363082,NaN,NaN,NaN,2015-12-21 10:41:00,2016-10-01,-285.0
42,1109139006,NaN,NaN,CA,201608.0,NaN,HOND,NaN,GO,14006 RIVERSIDE DR,...,HANDICAP/NO DP ID,363.0,34.157607,-118.438092,NaN,NaN,NaN,2015-12-19 15:50:00,2016-08-01,-226.0
43,1109455266,NaN,NaN,NY,NaN,NaN,TOYO,PA,WH,3200 CANYON DR,...,PK IN PROH AREA,68.0,34.126932,-118.311225,NaN,NaN,NaN,2015-12-26 18:15:00,NaT,NaN


In [81]:
# Imports
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import *

# df = gpd.read_file('sample_data.geojson')
# df['geometry'] = df['geometry'].apply(lambda x: WKTElement(x.wkt, srid=4326))
# df.drop('ticket_number', axis=1, inplace=True)

# Use 'dtype' to specify column's type
# For the geom column, we will use GeoAlchemy's type 'Geometry'
df.drop('Ticket number', axis=1).to_sql('test', engine, if_exists='replace', index=True, 
                         dtype={'issue_date': Date,
                            'fine_amount':Integer, 
                            'geometry': Geometry('POINT', srid=4326)})


In [ ]:
df_csv = pd.read_csv('Parking_Citations.csv')

In [ ]:
df_csv.head()

In [ ]:
# Imports

from sqlalchemy import *

# df = gpd.read_file('sample_data.geojson')
# df['geometry'] = df['geometry'].apply(lambda x: WKTElement(x.wkt, srid=4326))
# df.drop('ticket_number', axis=1, inplace=True)

# For the geometry column, we will use GeoAlchemy's type 'Geometry'
df.drop('Ticket number', axis=1).to_sql('citations', engine, if_exists='replace', index=False, 
                      dtype={'Datetime': DateTime,
                        'Fine_amount':Numeric, 
                        'Latitude':Float,
                        'Longitude': Float,
                        'Plate Expiry Date': Date
                        })